In [ ]:
#train.py
import math
import json
import matplotlib.pyplot as plot

#changeable settings
learningRate0 = 1
learningRate1 = 0.00007
error = 1e-6
scaling = 1000

# variables
file_contents = ""
file_milage = []
file_price = []
m = 0 # m is the total number of entries in the dataset cuz we wanna know the average, which is sum / total entries
theta0 = 0
theta1 = 0
r_squared_arr = []
mse_arr = []

def calc_estimate_price(milage):
	return theta0 + (theta1 * milage)

def calc_tmpTheta0():
	summation = 0
	for i in range(m):
		summation += calc_estimate_price(file_milage[i] / scaling) - file_price[i]
	return learningRate0 * (1/m) * summation

def calc_tmpTheta1():
	summation = 0
	for i in range(m):
		summation += (calc_estimate_price(file_milage[i] / scaling) - file_price[i]) * (file_milage[i] / 1000)
	return learningRate1 * (1/m) * summation


def process_file_contents():
	global m
	# split file contents into each line
	lines = file_contents.split('\n')
	if len(lines) <= 0:
		raise Exception("Error: File empty!!")
	
	#start from 1 to skip header
	for i in range(1, len(lines)):
		line_parts = lines[i].split(',')
		if len(line_parts) > 1:
			file_milage.append(float(line_parts[0].strip()))
			file_price.append(float(line_parts[1].strip()))
	
	m = len(file_milage)

def readfile():
	global file_contents
	file = open("data.csv")
	file_contents = file.read()
	file.close()

def train_model():
	global theta0, theta1, mse

	while True:

		tmpTheta0 = calc_tmpTheta0()
		tmpTheta1 = calc_tmpTheta1()

		if math.isnan(tmpTheta0) or math.isnan(tmpTheta1):
			print("NaN detected! Stopping calculation...")
			break

		if abs(tmpTheta0) < error and abs(tmpTheta1) < error:
			break
	
		theta0 -= tmpTheta0
		theta1 -= tmpTheta1

		calculate_r_squared()
		calculate_mse()

	theta1 /= scaling

def write_theta():
	params = {
		"theta0": theta0,
		"theta1": theta1,
		"mse": mse_arr[-1],
		"R2 error": r_squared_arr[-1]
	}

	try:
		f = open("params.json", "w")
		f.write(json.dumps(params))
		f.close()
	except Exception:
		print("Failed to save params")

# bonus
def plot_milage_vs_price():
	#scatter plot
	plot.scatter(file_milage, file_price, label='Actual data')
    
    #linear regression line
	min_milage = min(file_milage)
	max_milage = max(file_milage)
	x_line = []
	y_line = []
	for milage in range(int(min_milage), int(max_milage), 1000): #calculate every 1000km
		x_line.append(milage)
		y_line.append(theta0 + (theta1 * milage))
    
	plot.plot(x_line, y_line, color="red", label='Regression line')
	plot.xlabel('Mileage (km)')
	plot.ylabel('Price ($)')
	plot.title('Price vs Mileage - Linear Regression')
	plot.legend()
	plot.show()

def plot_rs_squared():
	plot.plot(range(len(r_squared_arr)), r_squared_arr, label='R2 Linear Regression Error')
	plot.xlabel('Iteration')
	plot.ylabel('R2 error')
	plot.title('R2 Linear Regression Error')
	plot.legend()
	plot.show()

def plot_mse():
	plot.plot(range(len(mse_arr)), mse_arr, label='MSE Linear Regression')
	plot.xlabel('Iteration')
	plot.ylabel('MSE')
	plot.title('MSE')
	plot.legend()
	plot.show()

def calculate_r_squared():
    mean_price = sum(file_price) / m

    ss_tot = 0
    for price in file_price:
        ss_tot += (price - mean_price) ** 2

    ss_res = 0
    for i in range(m):
        predicted = theta0 + (theta1 * (file_milage[i] / scaling))
        ss_res += (file_price[i] - predicted) ** 2
    
    r_squared = 1 - (ss_res / ss_tot)
    r_squared_arr.append(r_squared)

def calculate_mse():
	global mse_arr
	mse = 0
	for i in range(m):
		predicted_price = theta0 + (theta1 * (file_milage[i] / scaling))
		error = predicted_price - file_price[i]
		mse += error ** 2
	mse /= m
	mse_arr.append(mse)


if __name__ == "__main__":
	readfile()
	process_file_contents()
	train_model()
	write_theta()
	plot_milage_vs_price()
	plot_rs_squared()
	plot_mse()

In [ ]:
#predict.py
import json

theta0 = 0
theta1 = 0

def get_theta():
	global theta0, theta1
	try:
		file = open("params.json", "r")
		params_obj = json.load(file)
		theta0 = params_obj["theta0"]
		theta1 = params_obj["theta1"]
	except Exception as e:
		print("Error: unable to open file (prediction will continue but theta will be 0)")

def estimate_price(milage):
	return theta0 + theta1 * milage

if __name__ == "__main__":
	milage_string = input("Enter your milage (units in km, NUMBERS ONLY): ")
	try:
		get_theta()
		result = estimate_price(float(milage_string))
		if float(milage_string) < 10000:
			print("Result might be inaccurate because milage given is too small")
		print(f"Price for milage: {milage_string}km is ${result}")
	except Exception as e:
		print("Error:" , e)

I ESCAPED ENGINEERING BY STUDYING CS NOW ITS BACK TO TRAUMATISE ME

**Wat is linear regression:**
Basically just drawing a best fit line through a set of given data points

**Gradient decent algorithm:**
mse = (1 / m ) * SUM( [actual - predicted] ^ 2)

normally in linear regression ppl use: (1 / 2m ) * SUM( [actual - predicted] ^ 2), added the 2 to make it more convenient to differentiate, since it doesnt change where the minimum point is, it doesnt rly affect the goal of this algo

partial differentiating the mse algo (using chain rule) gives us the two gradient equations, or the two tmpTheta0 equations given in the pdf
after that, to find the minimum point of the equation (where mse is the least ofc we wanna have as less errors as possible), we can calculae the gradient at each point, then MINUS the gradient from the theta (use graph to visualise alot easier)

why do this? because the sign of the gradient (+ or -), gives us the direction we wanna move in, and the magnitude shows how far we are away from the point. For example, if we are very far on the left of the min point, the gradient will be a big -ve number, by minusing our current theta wif the big negative gradient, we move our point significantly to the right (closer to min point), same thing applies if we are on the right side of the min point. 

So when do we stop calculating? We stop when gradient ~ 0 (impossible to reach ngam ngam 0) where gradient is more or less horizontal

**Explaining the functions in the pdf:**

First function:
Linear function (y = mx + c)
ESTIMATES the price only

Second and third function:
Partial derivatives of the mse function, which is a cone / bowl shaped 3D graph (since mse depends on both thetas) with a minimum point where cost is the least


**Error calculation equations:**

R2 error:
1 - [ SUM(actual - predicted) / SUM(actual - mean)]

IF prediction is perfect: R2 = 1 (cuz actual - predicted = 0)
IF prediction is very bad: R2 < 0 (actual - predicted >>>> actual - mean)
IF prediction is meh: R2 < 1 (actual - predicted =/= 0 but < actual - mean)

mse (explained earlier above)

